# Smart Academic Advisor – RAG Pipeline

This notebook implements the core Retrieval-Augmented Generation (RAG) pipeline for the Smart Academic Advisor project.

We will:
- Load the processed academic dataset
- Prepare text fields for semantic embeddings
- (Later) Build a vector database and implement semantic search


### Step 1: Install required Python libraries

We install the libraries needed for embeddings, vector search, and data handling.


In [ ]:
!pip install -q sentence-transformers faiss-cpu numpy pandas


### Step 2: Mount Google Drive and load the processed dataset

We mount Google Drive and load the academic advisor dataset from its JSON file into a Python list.


In [ ]:
# Load the academic advisor dataset used in the main RAG notebook
from google.colab import drive
drive.mount('/content/drive')

data_path = "/content/drive/MyDrive/DAB_RAG_ZakyProject/data/processed/academic_advisor_rag_dataset.json"

with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print("Dataset loaded ✅")
print("Total records:", len(data))
print("First record keys:", data[0].keys())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset loaded ✅
Total records: 5069
First record keys: dict_keys(['id', 'source_file', 'university', 'catalog_label', 'section', 'section_chunk_index', 'category', 'program', 'college', 'degree', 'level', 'text'])


### Step 3: Convert the JSON data into a Pandas DataFrame

We convert the list of JSON records into a DataFrame to make inspection and filtering easier.


In [ ]:
import pandas as pd

# Convert list of dicts to DataFrame
df = pd.DataFrame(data)

print("DataFrame shape:", df.shape)
print("Columns:", df.columns.tolist())

# Show the first 3 rows
df.head(3)


DataFrame shape: (5069, 14)
Columns: ['id', 'source_file', 'university', 'catalog_label', 'section', 'section_chunk_index', 'category', 'program', 'college', 'degree', 'level', 'text', 'support_type', 'topic']


,id,source_file,university,catalog_label,section,section_chunk_index,category,program,college,degree,level,text,support_type,topic
0,1,csun_2025_2026_catalog,"California State University, Northridge",2025-2026 University Catalog,OEBPS/accounting.xhtml,0.0,general_academic,Accounting,David Nazarian College of Business and Economics,B.S.,undergraduate,Accounting\nAccounting\nDavid Nazarian College...,NaN,NaN
1,2,csun_2025_2026_catalog,"California State University, Northridge",2025-2026 University Catalog,OEBPS/accounting.xhtml,1.0,general_academic,Accounting,David Nazarian College of Business and Economics,B.S.,undergraduate,The Bachelor of Science in Accountancy (BSA) d...,NaN,NaN
2,3,csun_2025_2026_catalog,"California State University, Northridge",2025-2026 University Catalog,OEBPS/accounting.xhtml,2.0,general_academic,Accounting,David Nazarian College of Business and Economics,B.S.,undergraduate,Academic advisement helps students meet their ...,NaN,NaN


### Step 4: Build the input text used for semantic embeddings

We create a helper function that combines important metadata (program, degree, level, category) with the main text.
This richer context helps the embedding model capture academic meaning better.


In [ ]:
def build_embedding_text(row):
    """
    Build a combined text string from metadata + main text.
    This will be passed to the embedding model.
    """
    parts = []

    # Add high-level academic context if available
    if "program" in row and pd.notna(row["program"]):
        parts.append(f"Program: {row['program']}")
    if "degree" in row and pd.notna(row["degree"]):
        parts.append(f"Degree: {row['degree']}")
    if "level" in row and pd.notna(row["level"]):
        parts.append(f"Level: {row['level']}")
    if "category" in row and pd.notna(row["category"]):
        parts.append(f"Category: {row['category']}")

    # Add the main text field (core content)
    main_text = row.get("text", "")
    parts.append(str(main_text))

    # Join all parts into a single string
    return "\n".join(parts)

# Apply the function to all rows to create the embedding corpus
texts_for_embedding = [build_embedding_text(row) for _, row in df.iterrows()]

print("Total texts for embedding:", len(texts_for_embedding))
print("\nSample combined text:\n")
print(texts_for_embedding[0][:800])


Total texts for embedding: 5069

Sample combined text:

Program: Accounting
Degree: B.S.
Level: undergraduate
Category: general_academic
Accounting
Accounting
David Nazarian College of Business and Economics
Department of Accounting
Chair: Rishma Vedd
Bookstein Hall (BB) 3123
(818) 677-2461
Master of Professional Accountancy
Director: Rafael Efrat
Bookstein Hall (BB) 3123
(818) 677-2461
Master of Science in Taxation
Bookstein Chair in Taxation: Rafael Efrat
Bookstein Hall (BB) 3123
(818) 677-5488
EY Center for Careers in Accounting
Director: Gladys Polio
Bookstein Hall (BB) 2224
(818) 677-2979
Faculty
Katie Boylen, Keji Chen, Manuela Dantas, Michael E. Doron, Kiren Dosanjh-Zucker, Rafi Efrat, Monica Gianni, Young-Won Her, Yuan Yuan Lu, Joon Seok Moon, Rishma Vedd, Dongyi Wang, Sung Wook Yoon, Jun Zhan
Emeritus Faculty
Dhia D. Alhashim, Shahid L


### Step 5: Load the embedding model and generate vector representations

In this step, we:
1. Load a Sentence-Transformer model (`all-MiniLM-L6-v2`).
2. Encode all dataset texts (`texts_for_embedding`) into dense vector embeddings.
3. Normalize the embeddings so we can use cosine similarity (via inner product in FAISS) later.


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# 1) Choose and load the embedding model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
print(f"Loading embedding model: {model_name}")
embedder = SentenceTransformer(model_name)

# 2) Encode all texts into embeddings
print("Encoding texts into embeddings...")
embeddings = embedder.encode(
    texts_for_embedding,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

# 3) Inspect the embeddings matrix
print("Embeddings shape:", embeddings.shape)
print("Example vector (first record):")
print(embeddings[0][:10])


Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
Encoding texts into embeddings...


Batches:   0%|          | 0/80 [00:00<?, ?it/s]

Embeddings shape: (5069, 384)
Example vector (first record):
[ 0.0074278   0.00478056 -0.01619479 -0.02286152  0.00110091  0.0632113
  0.06548335  0.02125779  0.04273406  0.05182181]


### Step 6: Build a FAISS vector index for fast semantic search

In this step, we:
1. Create a FAISS index using the embeddings dimension.
2. Use `IndexFlatIP` (inner product) together with normalized embeddings to approximate cosine similarity.
3. Add all embeddings to the index so we can perform fast nearest-neighbor search later.


In [ ]:
import faiss

# 1) Get the dimensionality of the embedding vectors
dim = embeddings.shape[1]
print("Embedding dimension:", dim)

# 2) Create a FAISS index with inner product (works as cosine similarity with normalized vectors)
index = faiss.IndexFlatIP(dim)

# 3) Add all embeddings to the index
index.add(embeddings)

print("Total vectors in FAISS index:", index.ntotal)

# Quick sanity check: index size should equal number of rows in the dataset
assert index.ntotal == len(df), "Mismatch between index size and DataFrame length!"
print("FAISS index is consistent with the dataset ✅")


Embedding dimension: 384
Total vectors in FAISS index: 5069
FAISS index is consistent with the dataset ✅


### Step 7.1: Define a helper function to filter rows by metadata (optional)

We sometimes want to restrict search results to a specific subset of the catalog.
For example:
- Only `program = "Accounting"`
- Only `level = "undergraduate"`
- Only a certain `category` (e.g., "graduation_requirements")

This helper function returns the indices of rows that match a given filter dictionary.


In [ ]:
import numpy as np

def filter_indices(df, filters=None):
    """
    Return the indices of DataFrame rows that match the given filters.

    filters: dict like {"program": "Accounting", "level": "undergraduate"}
    If filters is None or empty, all row indices are returned.
    """
    if not filters:
        return np.arange(len(df))

    mask = np.ones(len(df), dtype=bool)

    for key, value in filters.items():
        if key not in df.columns:
            # Ignore filters that don't correspond to a column
            continue

        # Simple exact-match filter (can be improved later if needed)
        col_values = df[key].astype(str).fillna("")
        mask &= (col_values == str(value))

    return np.where(mask)[0]


### Step 7.2: Define the main semantic search function

This function will:
1. Encode the user's query into an embedding.
2. Optionally filter the dataset using metadata (program, level, category, etc.).
3. Perform a nearest-neighbor search in the embedding space (using FAISS or a temporary sub-index).
4. Return the top matching chunks along with their scores and metadata.


In [ ]:
def search(query, top_k=5, filters=None):
    """
    Perform a semantic search over the academic catalog.

    Args:
        query (str): The student's natural language question.
        top_k (int): Number of results to return.
        filters (dict, optional): Metadata filters, e.g.
            {"program": "Accounting", "level": "undergraduate"}

    Returns:
        List[dict]: A list of search results with score, metadata and text.
    """
    # 1) Encode the query into an embedding vector
    q_emb = embedder.encode(
        [query],
        convert_to_numpy=True,
        normalize_embeddings=True
    )

    # 2) Get candidate indices based on filters
    candidate_indices = filter_indices(df, filters)

    if len(candidate_indices) == 0:
        print("No documents match the metadata filters.")
        return []

    # If there are no filters (or all rows allowed), we can search over the full index
    if filters is None or len(filters) == 0 or len(candidate_indices) == len(df):
        # Use the global FAISS index directly
        D, I = index.search(q_emb, min(top_k, index.ntotal))

        results = []
        for score, idx in zip(D[0], I[0]):
            row = df.iloc[idx].to_dict()
            results.append({
                "score": float(score),
                "global_idx": int(idx),
                "id": row.get("id"),
                "program": row.get("program"),
                "category": row.get("category"),
                "degree": row.get("degree"),
                "level": row.get("level"),
                "text": row.get("text"),
            })
        return results

    # 3) If filters are applied, we build a temporary sub-index for those rows only
    sub_embs = embeddings[candidate_indices]

    sub_index = faiss.IndexFlatIP(dim)
    sub_index.add(sub_embs)

    # 4) Perform the search on the sub-index
    D, I = sub_index.search(q_emb, min(top_k, len(candidate_indices)))

    # 5) Map local indices back to global indices and build result objects
    results = []
    for score, local_idx in zip(D[0], I[0]):
        global_idx = int(candidate_indices[local_idx])
        row = df.iloc[global_idx].to_dict()
        results.append({
            "score": float(score),
            "global_idx": global_idx,
            "id": row.get("id"),
            "program": row.get("program"),
            "category": row.get("category"),
            "degree": row.get("degree"),
            "level": row.get("level"),
            "text": row.get("text"),
        })
    return results


### Step 7.3: Test the semantic search with example academic questions

We now test:
- A general question about admission requirements
- A question about total units required to graduate

This helps verify that the retriever is returning relevant catalog chunks.

Now we refine Query 2 by restricting results to the `graduation_requirements` category for the Accounting undergraduate program.


In [ ]:
# Example 1: General admission question
query_1 = "What are the admission requirements for the Accountancy bachelor program?"
results_1 = search(query_1, top_k=3)

print("=== Query 1:", query_1)
for r in results_1:
    print("\n" + "="*80)
    print(f"Score: {r['score']:.3f}")
    print(f"Program: {r['program']} | Category: {r['category']} | Level: {r['level']}")
    print(r['text'][:500], "...")


# Example 2: Question with filters (only undergraduate Accounting)
query_2 = "How many total units are required to graduate from the B.S. in Accountancy?"

results_2 = search(
    query_2,
    top_k=5,
    filters={
        "program": "Accounting",
        "level": "undergraduate",
        "category": "graduation_requirements"
    }
)

print("\n\n=== Refined Query 2:", query_2)
for r in results_2:
    print("\n" + "="*80)
    print(f"Score: {r['score']:.3f}")
    print(f"Program: {r['program']} | Category: {r['category']} | Level: {r['level']}")
    print(r['text'][:600], "...")



=== Query 1: What are the admission requirements for the Accountancy bachelor program?

Score: 0.730
Program: Accounting | Category: admissions_general | Level: undergraduate
An overall and CSUN GPA of 3.2 or higher for continuing CSUN students or an overall GPA of 3.2 or higher for first-semester transfer students. A minimum GPA of 3.0 shall satisfy this requirement if openings are available.
Declared as a Pre-Accountancy major.
Completion of a minimum of 60 units of college work (junior class standing). For transfer students, the units must be CSU transferable.
Admission to the Accountancy Major
Upon completion of ACCT 350 with a grade of &#8220;C&#8221; or higher, ...

Score: 0.728
Program: Accounting | Category: admissions_general | Level: undergraduate
An overall and CSUN GPA of 3.2 or higher for continuing CSUN students or an overall GPA of 3.2 or higher for first semester transfer students. A minimum GPA of 3.0 shall satisfy this requirement if openings are available.
Declared a

### Step 8.1: Build a helper to format retrieved chunks into a context block

This function takes the top retrieved chunks and combines them into a structured context.
This context will be used to simulate RAG-style answers without calling an external LLM.


In [ ]:
import html

def build_context_from_results(results, max_chars=2000):
    """
    Combine the top retrieved chunks into a single context string.

    - Decodes HTML entities (e.g., &#8220; → “).
    - Adds a small header before each chunk.
    """
    context_parts = []
    total_len = 0

    for i, r in enumerate(results, start=1):
        header = f"[Source {i}] Program: {r.get('program')} | Category: {r.get('category')} | Level: {r.get('level')}"
        text = str(r.get("text", "")).strip()
        text = html.unescape(text)

        block = header + "\n" + text + "\n"
        if total_len + len(block) > max_chars:
            break

        context_parts.append(block)
        total_len += len(block)

    if not context_parts:
        return "No relevant catalog entries found."

    return "\n\n".join(context_parts)


### Step 8.2: Implement a mock RAG answer generator (no external LLM)

This function:
1. Uses `search()` to retrieve the most relevant chunks.
2. Builds a readable context string from those chunks.
3. Returns a structured text response combining:
   - The original student question
   - A short template-style answer
   - The raw catalog snippets as "sources"


In [ ]:
def mock_rag_answer(question, top_k=5, filters=None):
    """
    A simple mock RAG answer generator (no external LLM).
    It retrieves relevant chunks and returns a structured answer string.
    """
    # 1) Retrieve top matching chunks
    results = search(question, top_k=top_k, filters=filters)

    if not results:
        return (
            "Question:\n"
            f"{question}\n\n"
            "Answer:\n"
            "I could not find any relevant information in the catalog for this question."
        )

    # 2) Build context from the retrieved chunks
    context = build_context_from_results(results)

    # 3) Compose a simple answer template
    answer_lines = []
    answer_lines.append("Question:")
    answer_lines.append(question)
    answer_lines.append("\nAnswer (based on catalog information):")
    answer_lines.append(
        "Below is the most relevant information retrieved from the official university catalog. "
        "You can read it and extract the exact details needed."
    )
    answer_lines.append("\nSources (catalog snippets):")
    answer_lines.append(context)

    return "\n".join(answer_lines)


In [ ]:
import os

os.makedirs("/content/drive/MyDrive/rag_artifacts", exist_ok=True)
print("Folder created ✅")


Folder created ✅


In [ ]:
import faiss
import json


faiss.write_index(index, "/content/drive/MyDrive/DAB_RAG_ZakyProject/rag_artifacts/academic_faiss.index")
print("FAISS index saved ✅")


config = {
    "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
    "embedding_dim": 384,
    "num_records": len(data)
}
with open("/content/drive/MyDrive/DAB_RAG_ZakyProject/rag_artifacts/rag_config.json", "w") as f:
    json.dump(config, f, indent=2)

print("Config saved ✅")


FAISS index saved ✅
Config saved ✅
